In [77]:
import json
import pandas as pd
from datetime import datetime
import numpy as np
import numpy as np
import math
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
import plotly.plotly as py
import cufflinks as cf
cf.go_online()

inpExifFl = "../data/imgs_exif_data_full.json"

with open(inpExifFl,"r") as inpJsonFl:
    exifJsonObj = json.load(inpJsonFl)

## Explaratory data analysis 
Summary stasitics and distributions

In [28]:
exifDf = pd.DataFrame(exifJsonObj).transpose()

exifDf['datetime'] = exifDf['datetime'].apply(lambda x : datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
exifDf['day'] = exifDf.datetime.apply(lambda x : x.day)
exifDf['hour'] = exifDf.datetime.apply(lambda x : x.hour)
exifDf.drop(['size','datetime'],1,inplace=True)
exifDf.head()

,height,lat,long,orientation,width,day,hour
1,4000,-1.3514,36.8002,1,6000,1,14
10,4000,-1.36709,36.782,1,6000,1,15
100,4000,-1.37325,36.8003,1,6000,1,16
1000,3072,-1.34968,36.806,1,4608,1,17
1001,3072,-1.34968,36.806,1,4608,1,17


In [98]:
trace1 = go.Histogram(
        x=exifDf['height']
    )

trace2 = go.Histogram(
        x=exifDf['width']
    )

trace3 = go.Histogram(
        x=exifDf['lat']
    )

trace4 = go.Histogram(
        x=exifDf['long']
    )

trace5 = go.Histogram(
        x=exifDf['orientation']
    )

trace6 = go.Histogram(
        x=exifDf['day']
    )

trace7 = go.Histogram(
        x=exifDf['hour']
    )

fig = tools.make_subplots(rows=4, cols=2, subplot_titles=('height','width','lat','long','orientation','day','hour'))

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)
fig.append_trace(trace5, 3, 1)
fig.append_trace(trace6, 3, 2)
fig.append_trace(trace7, 4, 1)

fig['layout'].update(showlegend=False, height=1000, width=1000, title='Data distributions')
plot(fig, filename='../FinalResults/ExplaratoryDataAnalysis_Distributions.html')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]
[ (3,1) x5,y5 ]  [ (3,2) x6,y6 ]
[ (4,1) x7,y7 ]  [ (4,2) x8,y8 ]



'file:///Users/sreejithmenon/Google Drive/CodeBase/AWESOME/FinalResults/ExplaratoryDataAnalysis_Distributions.html'

## Combining result data

In [60]:
from numpy import arange,array,ones
from scipy import stats
xi = arange(0,9)
A = array([ xi, ones(9)])

# (Almost) linear sequence
y = [19, 20, 20.5, 21.5, 22, 23, 23, 25.5, 24]

# Generated linear fit
slope, intercept, r_value, p_value, std_err = stats.linregress(xi,y)
line = slope*xi+intercept

# Creating the dataset, and generating the plot
trace1 = go.Scatter(
                  x=xi, 
                  y=y, 
                  mode='markers',
                  marker=go.Marker(color='rgb(255, 127, 14)'),
                  name='Data'
                  )

trace2 = go.Scatter(
                  x=xi, 
                  y=line, 
                  mode='lines',
                  marker=go.Marker(color='rgb(31, 119, 180)'),
                  name='Fit'
                  )


In [48]:
resultsDf = pd.DataFrame.from_csv("../FinalResults/resultsExpt2RankList_Tags.csv")

resultsDf = pd.DataFrame(resultsDf['Proportion'])
resultsDict = resultsDf.to_dict(orient="index")

expt2Results = {}

for gid in resultsDict:
    expt2Results[str(gid)] = exifJsonObj[str(gid)]
    expt2Results[str(gid)].update(resultsDict[gid])

In [51]:
expt2ResultsDf = pd.DataFrame(expt2Results).transpose()

expt2ResultsDf['datetime'] = expt2ResultsDf['datetime'].apply(lambda x : datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
expt2ResultsDf['day'] = expt2ResultsDf.datetime.apply(lambda x : x.day)
expt2ResultsDf['hour'] = expt2ResultsDf.datetime.apply(lambda x : x.hour)
expt2ResultsDf.drop(['size','datetime'],1,inplace=True)
expt2ResultsDf.head()

,Proportion,height,lat,long,orientation,width,day,hour
10,20,4000,-1.36709,36.782,1,6000,1,15
1000,90,3072,-1.34968,36.806,1,4608,1,17
1003,20,3072,-1.34965,36.8062,1,4608,1,17
1005,80,3072,-1.34965,36.8062,1,4608,1,17
101,20,4000,-1.37325,36.8003,1,6000,1,16


In [97]:
def linearRegressPlot(resultDf,xFtr,yStr):
    xi = np.array(resultDf[xFtr])
    y = list(resultDf[yStr])
    slope, intercept, r_value, p_value, std_err = stats.linregress(xi,y)
    line = slope*xi+intercept
    layout= go.Layout(
            title= "Linear line fit for regression using %s" %xFtr,
            showlegend=False,
            xaxis= dict(
                title= xFtr,
                ticklen= 5,
                zeroline= True,
                gridwidth= 2
            ),
            yaxis=dict(
                title= yStr,
                ticklen= 5,
                gridwidth= 2,
                #range=range
            )
        )

    trace1 = go.Scatter(
                x = xi,
                y = y,
                mode = 'markers'
        )

    trace2 = go.Scatter(
                x=xi, 
                y=line, 
                mode='lines',
                name='Fit'
                      )

    data = [trace1,trace2]

    fig = dict(data=data,layout=layout)
    figMain = py.iplot(fig,filename= str("linear_fit_%s" %xFtr))
    print("r_value, p_value" ,(r_value,p_value))
    print(figMain.embed_code)
linearRegressPlot(expt2ResultsDf,'orientation','Proportion')

r_value, p_value (0.013103869450779954, 0.59941570062121841)
<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~kamblenoopur/25.embed" height="525px" width="100%"></iframe>


In [61]:
xi = np.array(expt2ResultsDf['height'])
y = list(expt2ResultsDf['Proportion'])
slope, intercept, r_value, p_value, std_err = stats.linregress(xi,y)
slope, intercept, r_value, p_value, std_err

(-0.0014937529173252762,
 54.18473446939938,
 -0.051100111877733521,
 0.040414106564588367,
 0.000728251211858464)

array([48.20972280009828, 49.59592550737613, 49.59592550737613, ...,
       49.59592550737613, 49.59592550737613, 49.59592550737613], dtype=object)